# 학습된 에이전트 시각화 진행을 위한 노트북

### 필요 패키지 Import

In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd

# 조이스틱 환경 삽입
from horcrux_terrain_v1.envs import PlaneJoyWorld
from horcrux_terrain_v1.envs import PlaneWorld

# Ray 패키지 삽입
import ray
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.sac import SACConfig

from ray.tune.registry import register_env

### Ray 실행

In [ ]:
import socket
import psutil

conn_ip = ""
interfaces = psutil.net_if_addrs()
for interface_name, addresses in interfaces.items():
    if "openvpn" in interface_name.lower() and "tap" in interface_name.lower():
        snicaddrs = interfaces[str(interface_name)]
        for addrfamily in snicaddrs:
            if addrfamily.family == socket.AF_INET:
                conn_ip = addrfamily.address

# 해당 init을 통해서 VPN을 통한 외부 접속 가능함.
ray.init(dashboard_host=conn_ip, dashboard_port=8265)

2025-02-27 12:48:26,290	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.9
Ray version:,2.36.1
Dashboard:,http://127.0.0.1:8265


(autoscaler +51s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +51s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.0769}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +1m26s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.0769}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +2m1s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.0769}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +2m36s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.0769}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m11s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.0769}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m

### Gym 환경 등록하기

In [3]:
env_config = {
    "forward_reward_weight": 6.5,
    "side_cost_weight": 2.0,
    "unhealthy_max_steps": 100,
    "healthy_reward": 0.5,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.01,
    "rotation_orientation_cost_weight": 1.2,
    "termination_reward": 0,
    "gait_params": (30, 30, 60, 60, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
}

# JoyWorld
register_env("joy-v1", lambda config: PlaneJoyWorld( forward_reward_weight=env_config["forward_reward_weight"], 
                                                     side_cost_weight=env_config["side_cost_weight"], 
                                                     unhealthy_max_steps=env_config["unhealthy_max_steps"],
                                                     healthy_reward=env_config["healthy_reward"], 
                                                     healthy_roll_range=env_config["healthy_roll_range"],
                                                     terminating_roll_range=env_config["terminating_roll_range"],
                                                     rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                     rotation_orientation_cost_weight=env_config["rotation_orientation_cost_weight"],
                                                     termination_reward=env_config["termination_reward"],
                                                     gait_params=env_config["gait_params"],
                                                     use_friction_chg=env_config["use_friction_chg"],
                                                     joy_input_random=env_config["joy_input_random"],
                                                   )
            )

# Plane
register_env("plane-v1", lambda config: PlaneWorld(forward_reward_weight=env_config["forward_reward_weight"], 
                                                 side_cost_weight=env_config["side_cost_weight"], 
                                                 unhealthy_max_steps=env_config["unhealthy_max_steps"], 
                                                 healthy_reward=env_config["healthy_reward"],
                                                 healthy_roll_range=env_config["healthy_roll_range"],
                                                 terminating_roll_range=env_config["terminating_roll_range"],
                                                 rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                 rotation_orientation_cost_weight=env_config["rotation_orientation_cost_weight"],
                                                 termination_reward=env_config["termination_reward"],
                                                 use_friction_chg=env_config["use_friction_chg"],
                                                 gait_params=env_config["gait_params"]))

### 학습 알고리즘 불러오기

In [4]:
algo = Algorithm.from_checkpoint('./LP/')

2025-02-27 12:49:03,636	WARNING deprecation.py:50 -- DeprecationWarning: `_enable_new_api_stack` has been deprecated. Use `AlgorithmConfig._enable_new_api_stack` instead. This will raise an error in the future!
/opt/anaconda3/envs/gdtor/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/anaconda3/envs/gdtor/lib/python3.12/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
 

KeyboardInterrupt: 